In [1]:
import pandas as pd
from pathlib import Path
import re
import warnings
import sys
import os
import seaborn as sns
sys.path.append(os.path.join(sys.path[0], '../src'))

from data_preprocessor import preprocess_data

warnings.filterwarnings("ignore")

In [2]:
sns.set_style("whitegrid")

In [3]:
def print_sample(sample):
    """Print one sample from the entire dataset."""

    print(f"Context:{sample['context']}")
    try:
        print(f"Question:{sample['Question']}")
    except KeyError:
        print(f"Question:{sample['question']}")
    try:
        print(f"Answer:{sample['Answer']}")
    except KeyError:
        print(f"Answer:{sample['answer']}")

In [4]:
BASE_DIR = Path('EDA_MedQuAD_context_answers_sum.ipynb').resolve().parent.parent
DATA_DIR = BASE_DIR.joinpath('data')
FILE_NAME = "MedQuAD_context_answers_sum.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [5]:
data = pd.read_csv(FILE, sep="\t", header=None)
data.columns = ["question", "context", "answer"]

print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1110 non-null   object
 1   context   1102 non-null   object
 2   answer    1110 non-null   object
dtypes: object(3)
memory usage: 26.1+ KB
None


,question,context,answer
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляет...,кератодермия с пушистыми волосами представляет...
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляет...,кератодермия с пушистыми волосами встречается ...
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляет...,"гены, которые приводят к кератодермии с пушист..."
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляет...,кератодермия с курчавыми волосами имеет аутосо...
4,что такое синдром кноблоха?,"изза аномалий стекловидного тела, сетчатки и м...","В США диагностирован синдром Кноблоха, характе..."
5,сколько людей страдает синдромом кноблоха?,"изза аномалий стекловидного тела, сетчатки и м...","Синдром кноблоха редкое заболевание, однако ра..."
6,"каковы генетические изменения, связанные с син...","изза аномалий стекловидного тела, сетчатки и м...",Мутации в гене col18a1 могут вызвать синдром к...
7,передается ли синдром кноблоха по наследству?,"изза аномалий стекловидного тела, сетчатки и м...",Человек с аутосомнорецессивным заболеванием на...
8,что такое колобома?,они могут выглядеть как выемки или пробелы в о...,В настоящее время можно было бы только видеть ...
9,сколько людей страдают от колобомы?,они могут выглядеть как выемки или пробелы в о...,Колобома часто встречается у 1 из 10 000 людей...


In [6]:
data[data["context"].isna()]

,question,context,answer
287,есть ли у вас информация о семейных проблемах...,NaN,Американские организации являются хорошими ист...
299,что такое артрит ресурсы? также называется: р...,NaN,Американская академия хирурговортопедов и фонд...
330,что такое являются бёрнс ресурсы? также назыв...,NaN,Американская ожоговая ассоциация www.ameriburn...
388,есть ли у вас информация о ресурсах также наз...,NaN,Местные и национальные группы поддержки можно ...
867,есть ли у вас информация о потеря ребенка рес...,NaN,"Родители, пережившие скорбь в сша родители, по..."
893,что такое цинга? также называется: дефицит ви...,NaN,"цинга — это заболевание, которое возникает при..."
927,есть ли у вас информация о функциональных тес...,NaN,Тесты печени и функциональные тесты крови явля...
964,что такое являются бесплодие ресурсы? также н...,NaN,Информация о бесплодии может быть опубликована...


In [7]:
print_sample(data.iloc[14])

Context:синдром lacrimoauriculodentodigital ladd генетическое заболевание, которое в основном поражает глаза, уши, рот и руки. пороки развития слезной системы, которые могут возникнуть при синдроме ladd, включают недоразвитие или отсутствие отверстия слезного протока на краю века слезные точки и закупорку канала носослезного канала, соединяющего внутренний угол глаза, где собираются слезы слезный мешок в полость носа. люди с синдромом ladd часто имеют маленькие, недоразвитые зубы с тонкой эмалью и передние зубы резцы в форме колышек. у людей с синдромом ladd могут также наблюдаться другие признаки и симптомы. у них могут возникнуть проблемы с почками, включающие затвердевание почек нефросклероз и скопление мочи в почках гидронефроз, что может ухудшить функцию почек. причины мутации в генах fgfr2, fgfr3 или fgf10 могут вызвать синдром ladd. в результате нарушается созревание и развитие клеток, а также нарушается формирование многих тканей, что приводит к появлению признаков и симптомов 

### Data Preprocessing for the entire dataframe

In [8]:
# Remove samples with NaNs
data.dropna(inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102 entries, 0 to 1109
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1102 non-null   object
 1   context   1102 non-null   object
 2   answer    1102 non-null   object
dtypes: object(3)
memory usage: 34.4+ KB


In [10]:
data = preprocess_data(data)
data

,question,context,answer
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляет...,кератодермия с пушистыми волосами представляет...
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляет...,кератодермия с пушистыми волосами встречается ...
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляет...,"гены, которые приводят к кератодермии с пушист..."
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляет...,кератодермия с курчавыми волосами имеет аутосо...
4,что такое синдром кноблоха?,"изза аномалий стекловидного тела, сетчатки и м...","в сша диагностирован синдром кноблоха, характе..."
...,...,...,...
1105,есть ли у вас информация об измерении темпера...,оставьте термометр во рту на 3 минуты или пока...,"если температура тела может быть высока, то эт..."
1106,что такое дефицит аденозинмонофосфатдезаминазы?,у многих больных дефицит амфдезаминазы не вызы...,"ученые считают, что дефицит амфдезаминазы може..."
1107,сколько людей страдают от дефицита аденозинмо...,у многих больных дефицит амфдезаминазы не вызы...,амфдезаминазы является одним из наиболее распр...
1108,"каковы генетические изменения, связанные с де...",у многих больных дефицит амфдезаминазы не вызы...,человек с дефицитом амфдезаминазы страдает от ...


In [11]:
data.index = [i for i in range(len(data))]

In [12]:
data.groupby(["question", "context"])["answer"].count()

question                                                   context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [13]:
# Remove extra information from the answer
# data.question = [f"{question.split('?')[0]}?" for question in data.question]
# data

In [14]:
data.iloc[160].question

' что такое целиакия вопросы питания?'

### Save Dataset

In [15]:
DATA_NAME = "MedQuAD_context_answer_sum_clean.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)

### Concatenate with original Dataset

In [16]:
# Check the distribution of password lengths of the question-answer for the original dataset and the augmented one

# Read initial dataset
FILE_NAME = "MedQuAD_clean.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

data_init = pd.read_csv(FILE, sep=",", index_col=0)
data_init.columns = ["question", "context", "answer", "answer_start"]
data_init.index = [i for i in range(len(data_init))]

print(data_init.info())
data_init.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1113 entries, 0 to 1112
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   question      1113 non-null   object
 1   context       1113 non-null   object
 2   answer        1113 non-null   object
 3   answer_start  1113 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 43.5+ KB
None


,question,context,answer,answer_start
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...,9
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...,9
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю...",2551
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...,4573
4,что такое синдром кноблоха?,"синдром кноблоха — редкое заболевание, характ...","синдром кноблоха — редкое заболевание, характе...",9
5,сколько людей страдает синдромом кноблоха?,"синдром кноблоха — редкое заболевание, характ...",синдром кноблоха — редкое заболевание. однако ...,9
6,"каковы генетические изменения, связанные с син...","синдром кноблоха — редкое заболевание, характ...",мутации в гене col18a1 могут вызвать синдром к...,1794
7,передается ли синдром кноблоха по наследству?,"синдром кноблоха — редкое заболевание, характ...",это состояние наследуется по аутосомно-рецесси...,3141
8,что такое колобома?,"колобома — это глазная аномалия, возникающая ...","колобома – это глазная аномалия, возникающая е...",9
9,сколько людей страдают от колобомы?,"колобома — это глазная аномалия, возникающая ...",колобома встречается примерно у 1 из 10 000 че...,9


In [17]:
data = pd.concat([data_init, data])
data

,question,context,answer,answer_start
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...,9.0
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...,9.0
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю...",2551.0
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...,4573.0
4,что такое синдром кноблоха?,"синдром кноблоха — редкое заболевание, характ...","синдром кноблоха — редкое заболевание, характе...",9.0
...,...,...,...,...
1097,есть ли у вас информация об измерении темпера...,оставьте термометр во рту на 3 минуты или пока...,"если температура тела может быть высока, то эт...",NaN
1098,что такое дефицит аденозинмонофосфатдезаминазы?,у многих больных дефицит амфдезаминазы не вызы...,"ученые считают, что дефицит амфдезаминазы може...",NaN
1099,сколько людей страдают от дефицита аденозинмо...,у многих больных дефицит амфдезаминазы не вызы...,амфдезаминазы является одним из наиболее распр...,NaN
1100,"каковы генетические изменения, связанные с де...",у многих больных дефицит амфдезаминазы не вызы...,человек с дефицитом амфдезаминазы страдает от ...,NaN


In [18]:
data.groupby(["question", "context"])["answer"].count()

question                                                   context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [19]:
data.index = [i for i in range(len(data))]

In [20]:
DATA_NAME = "MedQuAD_context_answer_sum_concat.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)